In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-3e63a993-41e8-47d9-6b2c-2a5f9f227925)


In [2]:
!ls

Firm_relation_extraction_LUKE.ipynb			      onstart.sh
lr_find_temp_model_75555ed7-5ed1-4cb6-bab9-010cd54dd15d.ckpt  wandb
onstart.log


In [3]:
# !rm -r LUKE # delete old runs

rm: cannot remove 'LUKE': No such file or directory


In this notebook, we are going to fine-tune [`LukeForEntityPairClassification`](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) on a supervised **relation extraction** dataset.

The goal for the model is to predict, given a sentence and the character spans of two entities within the sentence, the relationship between the entities.

* Paper: https://arxiv.org/abs/2010.01057
* Original repository: https://github.com/studio-ousia/luke

In [4]:
!pip install -q transformers 
!pip install -q pandas
!pip install -q sklearn
!pip install -q pytorch-lightning wandb

In [3]:
from transformers import LukeTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
rs = 42

## Read in data

Let's download the data from the web, hosted on Dropbox.

Each row in the dataframe consists of a news article. 

In [6]:
df = pd.read_pickle("https://www.dropbox.com/s/rdwg8d76ytqqgdy/training-data-29-03-22.pkl?dl=1")
df

,Date,source,document,firms,spans,rels
233892045,1985-01-01,Thomson SDC alliances - Deal Number 233892045,Accel Capital L.P. invested an undisclosed amo...,"[Accel Capital, Thalamus Electronics Inc]","[(0, 13), (53, 77)]","[StrategicAlliance, Manufacturing]"
216749045,1985-01-01,Thomson SDC alliances - Deal Number 216749045,Standard Microsystems Corp. and Mitsubishi Ele...,"[Standard Microsystems Corp, Mitsubishi Electr...","[(0, 26), (32, 56)]","[StrategicAlliance, Licensing]"
234987045,1985-01-01,Thomson SDC alliances - Deal Number 234987045,"Elf Technologies, Inc., the venture capital in...","[Elf Technologies, Santa Clara Systems]","[(0, 16), (169, 188)]",[StrategicAlliance]
216600045,1985-01-01,Thomson SDC alliances - Deal Number 216600045,Stauffer Chemical Co. has been licensed by Mit...,"[Stauffer Chemical Co, Himont Inc]","[(0, 20), (79, 89)]","[StrategicAlliance, Marketing, Manufacturing, ..."
2410877045,1985-01-01,Thomson SDC alliances - Deal Number 2410877045,The Romanian Government (RG) and Control Data ...,"[Romania, Control Data Corp]","[(4, 11), (33, 50)]","[JointVenture, Manufacturing]"
...,...,...,...,...,...,...
257823,NaT,Reuters News Dataset - Article ID http://www.b...,University of Notre Dame football radio anal...,"[Notre Dame, Notre Dame’s]","[(15, 25), (200, 212)]",[]
416616,NaT,Reuters News Dataset - Article ID http://www.b...,"Schaeffler AG, the industrial-bearing maker th...","[Schaeffler AG, Continental AG]","[(0, 13), (91, 105)]",[]
328325,NaT,Reuters News Dataset - Article ID http://www.r...,Barnes & Noble Inc ( BKS.N ) said on Friday it...,"[Barnes & Noble, Lynch]","[(0, 14), (113, 118)]",[]
121959,NaT,Reuters News Dataset - Article ID http://www.b...,(Corrects third paragraph to show output of 25...,"[First Solar Inc ., Edison International]","[(154, 171), (249, 269)]",[]


Shuffle the data:

In [8]:
df = df.sample(frac=1, random_state=rs)

This is the frequency of our relation types:



In [9]:
rel_frequencies = df.rels.explode().value_counts()
rel_frequencies

StrategicAlliance         83845
JointVenture              44556
Pending                   38916
Marketing                 22478
Manufacturing             19926
ResearchandDevelopment    13568
Licensing                 12857
TechnologyTransfer         9889
Supply                     5407
Exploration                3152
Terminated                 1578
Name: rels, dtype: int64

We don't want to consider the less relevant and less consistently applied labels of 'supply', 'exploration', and 'technology transfer'. Let's remove them:

In [10]:
df['rels'] = df.rels.apply(lambda rels: [rel for rel in rels if rel not in ['TechnologyTransfer', 'Supply', 'Exploration']])

In [11]:
rel_frequencies = df.rels.explode().value_counts()
rel_frequencies

StrategicAlliance         83845
JointVenture              44556
Pending                   38916
Marketing                 22478
Manufacturing             19926
ResearchandDevelopment    13568
Licensing                 12857
Terminated                 1578
Name: rels, dtype: int64

Let's one hot encode the relationship classes:

In [12]:
rel_label_names = rel_frequencies.index.to_list()

def label2ids(labels, rel_label_names=rel_label_names):
  return [1 if rel_name in labels else 0 for rel_name in rel_label_names]
df['rel_one_hot'] = df.rels.apply(label2ids)

def ids2labels(ids, rel_label_names=rel_label_names):
  labels = []
  for idx, label in enumerate(rel_label_names):
    if ids[idx] == 1:
      labels.append(label)
    
  return labels

df.sample(10)

,Date,source,document,firms,spans,rels,rel_one_hot
399232045,1994-03-21,Thomson SDC alliances - Deal Number 399232045,Mizuno Corp and the University of Electro-Comm...,"[Mizuno Corp, University of Electro-Commun]","[(0, 11), (20, 48)]","[StrategicAlliance, ResearchandDevelopment]","[1, 0, 0, 0, 0, 1, 0, 0]"
2930528045,2016-02-11,Thomson SDC alliances - Deal Number 2930528045,Monster Worldwide Inc and kununu GmbH planned ...,"[Monster Worldwide Inc, kununu GmbH]","[(0, 21), (26, 37)]","[JointVenture, Pending]","[0, 1, 1, 0, 0, 0, 0, 0]"
3417271045,2019-06-13,Thomson SDC alliances - Deal Number 3417271045,CareSource Management Group Co and NantHealth ...,"[CareSource Management Group Co, NantHealth Inc]","[(0, 30), (35, 49)]","[StrategicAlliance, Pending]","[1, 0, 1, 0, 0, 0, 0, 0]"
1005899045,2000-05-12,Thomson SDC alliances - Deal Number 1005899045,Vertical Computer Systems Inc (VCS) and iNetPu...,"[Vertical Computer Systems Inc, iNetPurchasing...","[(0, 29), (40, 58)]",[StrategicAlliance],"[1, 0, 0, 0, 0, 0, 0, 0]"
329691,NaT,Reuters News Dataset - Article ID http://www.b...,The Philippines is stepping up its battle ag...,"[Citigroup Inc., Nomura Holdings Inc.]","[(455, 469), (474, 494)]",[],"[0, 0, 0, 0, 0, 0, 0, 0]"
435409045,1994-11-14,Thomson SDC alliances - Deal Number 435409045,Toenec Corp and Beijing Guangyao International...,"[Toenec Corp, Beijing Guangyao International]","[(0, 11), (16, 46)]",[JointVenture],"[0, 1, 0, 0, 0, 0, 0, 0]"
459181045,1995-03-29,Thomson SDC alliances - Deal Number 459181045,"PPG Industries Inc, Mitsui & Co Ltd, and an un...","[PPG Industries Inc, Mitsui & Co Ltd]","[(0, 18), (20, 35)]","[JointVenture, Manufacturing]","[0, 1, 0, 0, 1, 0, 0, 0]"
3615840045,2020-07-28,Thomson SDC alliances - Deal Number 3615840045,Repsol SA and Nortegas Green Energy Solutions ...,"[Repsol SA, Nortegas Green Energy]","[(0, 9), (14, 35)]",[StrategicAlliance],"[1, 0, 0, 0, 0, 0, 0, 0]"
598008045,1996-09-12,Thomson SDC alliances - Deal Number 598008045,Eagle Finance Corp and Merdeka Park Holdings S...,"[Eagle Finance Corp, Merdeka Park Holdings Sdn...","[(0, 18), (23, 52)]",[JointVenture],"[0, 1, 0, 0, 0, 0, 0, 0]"
74092,NaT,Reuters News Dataset - Article ID http://www.b...,Nilesh Jasani quit Credit Suisse Group AG tod...,"[Credit Suisse Group AG, Jefferies Group Inc.]","[(20, 42), (58, 78)]",[],"[0, 0, 0, 0, 0, 0, 0, 0]"


To deal with the imbalance, we use weights in the loss function proportional to the inverse relative frequency of each class, normalizing by the frequency of the most common class. The resulting weights are as follows:

In [13]:
class_weights = 1/(rel_frequencies/rel_frequencies.iloc[0])
class_weights

StrategicAlliance          1.000000
JointVenture               1.881789
Pending                    2.154512
Marketing                  3.730092
Manufacturing              4.207819
ResearchandDevelopment     6.179614
Licensing                  6.521350
Terminated                53.133714
Name: rels, dtype: float64

The strategic alliance label is 6 times more frequent than the R&D label, therefore we weight the R&D class 6 times heavier in the loss function.

We can even increase the reweighting by exponentiating these weights:

In [14]:
IMBALANCE_REWEIGHTING = 1.3
class_weights = class_weights**IMBALANCE_REWEIGHTING
class_weights

StrategicAlliance           1.000000
JointVenture                2.274795
Pending                     2.712400
Marketing                   5.536501
Manufacturing               6.475513
ResearchandDevelopment     10.672096
Licensing                  11.445606
Terminated                174.977103
Name: rels, dtype: float64

Now examples with the R&D label are weighted 10 times heavier than those with the SA label.

In [15]:
class_weights = torch.tensor(class_weights.to_list()).to(device)

## Define the PyTorch dataset and dataloaders

Next, we define regular PyTorch datasets and corresponding dataloaders. In PyTorch, you need to define a `Dataset` class that inherits from `torch.utils.data.Dataset`, and you need to implement 3 methods: the `init` method (for initializing the dataset with data), the `len` method (which returns the number of elements in the dataset) and the `getitem()` method, which returns a single item from the dataset.

In our case, each item of the dataset consists of a sentence, the spans of 2 entities in the sentence, and a label of the relationship. We use `LukeTokenizer` (available in the Transformers library) to turn these into the inputs expected by the model, which are `input_ids`, `entity_ids`, `attention_mask`, `entity_attention_mask` and `entity_position_ids`.

For more information regarding these inputs, refer to the [docs](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) of `LukeForEntityPairClassification`.


Let's set our hyperparameters:

In [16]:
MAX_LEN = 128
LEARNING_RATE = 1e-05
BATCH_SIZE = 128
THRESHOLDS = [0.5 for rel in rel_label_names] # equal threshold for each class
MAX_EPOCHS = 1

In [17]:
THRESHOLDS = torch.tensor(THRESHOLDS).to(device)

In [18]:
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base", task="entity_pair_classification")

class RelationExtractionDataset(Dataset):
    """Relation extraction dataset."""

    def __init__(self, data):
        """
        Args:
            data : Pandas dataframe.
        """
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]

        text = item.document

        encoding = tokenizer(text, entity_spans=item.spans, padding="max_length", truncation=True, return_tensors="pt",
                            max_length=MAX_LEN)

        for k,v in encoding.items():
          encoding[k] = encoding[k].squeeze()

        encoding["label"] = torch.tensor(item.rel_one_hot)

        return encoding

Here we instantiate the class defined above with 3 objects: a training dataset, a validation dataset and a test set.

In [19]:
from sklearn.model_selection import train_test_split

train_size = 0.8
validation_size = 0.1
test_size = 0.1

train_df, test_df = train_test_split(df, test_size=1-train_size, random_state=rs, shuffle=True)
val_df, test_df = train_test_split(test_df, test_size=test_size/(test_size+validation_size), random_state=rs, shuffle=False)

print("FULL Dataset: {}".format(len(df)))
print("TRAIN Dataset: {}".format(len(train_df)))
print("TEST Dataset: {}".format(len(test_df)))
print("VALIDATION Dataset: {}".format(len(val_df)))

# define the dataset
train_dataset = RelationExtractionDataset(data=train_df)
valid_dataset = RelationExtractionDataset(data=val_df)
test_dataset = RelationExtractionDataset(data=test_df)

FULL Dataset: 264928
TRAIN Dataset: 211942
TEST Dataset: 26493
VALIDATION Dataset: 26493


In [20]:
train_dataset[0].keys()

dict_keys(['input_ids', 'entity_ids', 'entity_position_ids', 'attention_mask', 'entity_attention_mask', 'label'])

## Define a PyTorch LightningModule

Let's define the model as a PyTorch LightningModule. A `LightningModule` is actually an `nn.Module`, but with some extra functionality.

For more information regarding how to define this, see the [docs](https://pytorch-lightning.readthedocs.io/en/latest/?_ga=2.56317931.1395871250.1622709933-1738348008.1615553774) of PyTorch Lightning.

In [140]:
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

In [25]:
print(f'Free CUDA memory: {(torch.cuda.get_device_properties(0).total_memory-torch.cuda.memory_allocated(0))/1e9:.2f}GB')

Free CUDA memory: 25.45GB


In [26]:
from transformers import LukeForEntityPairClassification, AdamW
import pytorch_lightning as pl
from sklearn import metrics

class LUKE(pl.LightningModule):

    def __init__(self, lr=LEARNING_RATE, batch_size=BATCH_SIZE, thresholds=THRESHOLDS):
        super().__init__()
        self.model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-base", num_labels=len(rel_label_names))
        self.save_hyperparameters()

    def forward(self, input_ids, entity_ids, entity_position_ids, attention_mask, entity_attention_mask):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, entity_ids=entity_ids, 
                             entity_attention_mask=entity_attention_mask, entity_position_ids=entity_position_ids)
        return outputs
    
    def common_step(self, batch, batch_idx):
        labels = batch['label'].float()
        del batch['label']
        outputs = self(**batch)
        logits = outputs.logits
        
        criterion = torch.nn.BCEWithLogitsLoss(weight=class_weights) # multi-label classification with weighted classes
        loss = criterion(logits, labels)
        preds = (torch.sigmoid(logits)>self.hparams.thresholds).float()

        return {'loss': loss, 'preds': preds, 'labels': labels}
      
    def training_step(self, batch, batch_idx):
        output = self.common_step(batch, batch_idx)
        loss = output['loss']
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        output = self.common_step(batch, batch_idx)     
        loss = output['loss']
        self.log("val_loss", loss, on_epoch=True)
        
        preds = output['preds']
        labels = output['labels']
        
        return {"loss": loss, "preds": preds, "labels": labels}
    
    def validation_epoch_end(self, outputs):
            for i, output in enumerate(outputs):
                preds = output["preds"].detach().cpu().numpy()
                labels = output["labels"].detach().cpu().numpy()
                loss = output["loss"].mean()

                for idx, label_name in enumerate(rel_label_names):
                    label = [label[idx] for label in labels]
                    pred = [pred[idx] for pred in preds]

                    precision = metrics.precision_score(label, pred)
                    recall = metrics.recall_score(label, pred)
                    f1 = metrics.f1_score(label, pred)
                    self.log(f'val_precision_{label_name}', precision)
                    self.log(f'val_recall_{label_name}', recall)
                    self.log(f'val_f1_{label_name}', f1, prog_bar=True)
        
    def test_step(self, batch, batch_idx):
        output = self.common_step(batch, batch_idx)     
        loss = output['loss']
        self.log("test_loss", loss, on_epoch=True)
        
        preds = output['preds']
        labels = output['labels']
        
        return {"loss": loss, "preds": preds, "labels": labels}
    
    def test_epoch_end(self, outputs):
            for i, output in enumerate(outputs):
                preds = output["preds"].detach().cpu().numpy()
                labels = output["labels"].detach().cpu().numpy()
                loss = output["loss"].mean()

                for idx, label_name in enumerate(rel_label_names):
                    label = [label[idx] for label in labels]
                    pred = [pred[idx] for pred in preds]
                    precision = metrics.precision_score(label, pred)
                    recall = metrics.recall_score(label, pred)
                    f1 = metrics.f1_score(label, pred)
                    self.log(f'test_precision_{label_name}', precision)
                    self.log(f'test_recall_{label_name}', recall)
                    self.log(f'test_f1_{label_name}', f1, prog_bar=True)
        
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        return optimizer

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.hparams.batch_size, shuffle=True, num_workers=16)

    def val_dataloader(self):
        return DataLoader(valid_dataset, batch_size=self.hparams.batch_size, num_workers=16)

    def test_dataloader(self):
        return DataLoader(test_dataset, batch_size=self.hparams.batch_size, num_workers=16)

    
model = LUKE()

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForEntityPairClassification: ['embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LukeForEntityPairClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model

Let's train the model. We also use early stopping, to avoid overfitting the training dataset. We also log everything to Weights and Biases, which will give us beautiful charts of the loss and accuracy plotted over time.

If you haven't already, you can create an account on the [website](https://wandb.ai/site), then log in in a web browser, and run the cell below: 

In [27]:
import wandb

wandb.login()

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


True

In [28]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

wandb_logger = WandbLogger(project='LUKE')

# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    strict=False,
    verbose=False,
    mode='min'
)

checkpoint_callback = ModelCheckpoint(dirpath='LUKE')

trainer = Trainer(gpus=1, logger=wandb_logger, callbacks=[EarlyStopping(monitor='val_loss'), checkpoint_callback], 
                  max_epochs=MAX_EPOCHS, precision=16, 
                  stochastic_weight_avg=True, auto_lr_find=True, 
                  benchmark=True, deterministic=True,
                  val_check_interval=0.1, limit_train_batches=0.1)
                    # fast_dev_run=True
                    # limit_train_batches=0.25

Using 16bit native Automatic Mixed Precision (AMP)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:58: LightningDeprecationWarning: Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Finding the optimal learning rate:

In [ ]:
lr_finder = trainer.tuner.lr_find(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans 

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


...

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


In [26]:
lr_finder.suggestion()

2.2908676527677725e-05

Setting the learning rate to the optimal rate:

In [27]:
model.hparams.lr = lr_finder.suggestion()

Let's train the model:

In [28]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



  | Name  | Type                            | Params
----------------------------------------------------------
0 | model | LukeForEntityPairClassification | 274 M 
----------------------------------------------------------
274 M     Trainable params
0         Non-trainable params
274 M     Total params
549.029   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


Training: 0it [00:00, ?it/s]

...

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is 

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being s

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

In [29]:
trainer.test()

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1398: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /workspace/LUKE/epoch=0-step=1649.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /workspace/LUKE/epoch=0-step=1649.ckpt


Testing: 0it [00:00, ?it/s]

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


...

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision 

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_JointVenture': 0.982008727521514,
 'test_f1_Licensing': 0.8186806307389375,
 'test_f1_Manufacturing': 0.8338465278412275,
 'test_f1_Marketing': 0.764340621689146,
 'test_f1_Pending': 0.9324661993991683,
 'test_f1_ResearchandDevelopment': 0.724359310865089,
 'test_f1_StrategicAlliance': 0.9877974433310706,
 'test_f1_Terminated': 0.4345755693581781,
 'test_loss': 0.482481986284256,
 'test_precision_JointVenture': 0.9762852879044043,
 'test_precision_Licensing': 0.8864458944169089,
 'test_precision_Manufacturing': 0.8638284890096486,
 'test_precision_Marketing': 0.8344311643331251,
 'test_precision_Pending': 0.9409520948081314,
 'test_precision_ResearchandDevelopment': 0.7875513241455272,
 'test_precision_StrategicAlliance': 0.9932036466689815,
 'test_precision_Terminated': 0.44806763285024154,
 'test_recall_JointVenture': 0.9887626706813345,
 'test_recall_Licensing': 0.786

[{'test_loss': 0.482481986284256,
  'test_precision_StrategicAlliance': 0.9932036466689815,
  'test_recall_StrategicAlliance': 0.9827832040310484,
  'test_f1_StrategicAlliance': 0.9877974433310706,
  'test_precision_JointVenture': 0.9762852879044043,
  'test_recall_JointVenture': 0.9887626706813345,
  'test_f1_JointVenture': 0.982008727521514,
  'test_precision_Pending': 0.9409520948081314,
  'test_recall_Pending': 0.9277471308461672,
  'test_f1_Pending': 0.9324661993991683,
  'test_precision_Marketing': 0.8344311643331251,
  'test_recall_Marketing': 0.7232104161665112,
  'test_f1_Marketing': 0.764340621689146,
  'test_precision_Manufacturing': 0.8638284890096486,
  'test_recall_Manufacturing': 0.8204794802620888,
  'test_f1_Manufacturing': 0.8338465278412275,
  'test_precision_ResearchandDevelopment': 0.7875513241455272,
  'test_recall_ResearchandDevelopment': 0.7008286533648852,
  'test_f1_ResearchandDevelopment': 0.724359310865089,
  'test_precision_Licensing': 0.8864458944169089,
 

In [30]:
checkpoint_callback.best_model_path

'/workspace/LUKE/epoch=0-step=1649.ckpt'

In [32]:
!pip -q install dropbox

In [3]:
import dropbox
import os
from tqdm import tqdm

access_token = 'sl.BEuonkM5dN24jg8sgsMvxGGn_Sjdn0Is3DJpg2CRKpUgJQnkzkat6QU9OxNC59hnEK-JAACG4aGKNbNL5FiBfNRmtd1R6RpSM8Lb9Z720LFNpCuNKruvreOM3JE4qYDydyfXh9y8XTE'

file_from = 'LUKE/f4zydpek/checkpoints/epoch=4-step=8384.ckpt'
file_to = '/trained_models/29-3-22-17h-epoch=4-step=8384.ckpt'  # The full path to upload the file to, including the file name

def upload(
    access_token,
    file_path,
    target_path,
    timeout=900,
    chunk_size=4 * 1024 * 1024,
):
    dbx = dropbox.Dropbox(access_token, timeout=timeout)
    with open(file_path, "rb") as f:
        file_size = os.path.getsize(file_path)
        if file_size <= chunk_size:
            print(dbx.files_upload(f.read(), target_path))
        else:
            with tqdm(total=file_size, desc="Uploaded") as pbar:
                upload_session_start_result = dbx.files_upload_session_start(
                    f.read(chunk_size)
                )
                pbar.update(chunk_size)
                cursor = dropbox.files.UploadSessionCursor(
                    session_id=upload_session_start_result.session_id,
                    offset=f.tell(),
                )
                commit = dropbox.files.CommitInfo(path=target_path)
                while f.tell() < file_size:
                    if (file_size - f.tell()) <= chunk_size:
                        print(
                            dbx.files_upload_session_finish(
                                f.read(chunk_size), cursor, commit
                            )
                        )
                    else:
                        dbx.files_upload_session_append(
                            f.read(chunk_size),
                            cursor.session_id,
                            cursor.offset,
                        )
                        cursor.offset = f.tell()
                    pbar.update(chunk_size)

upload(
    access_token,
    file_from,
    file_to,
    timeout=900,
    chunk_size=100 * 1024 * 1024,
)

Uploaded: 3355443200it [04:07, 13533766.16it/s]                    

FileMetadata(client_modified=datetime.datetime(2022, 3, 29, 15, 40, 29), content_hash='a3dda202b0625ed24a81318cbb5005cd53468219babefa223bdc7d3fc2c24ffa', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:c181gNQpVocAAAAAAAAfXQ', is_downloadable=True, media_info=NOT_SET, name='29-3-22-17h-epoch=4-step=8384.ckpt', parent_shared_folder_id=NOT_SET, path_display='/trained_models/29-3-22-17h-epoch=4-step=8384.ckpt', path_lower='/trained_models/29-3-22-17h-epoch=4-step=8384.ckpt', property_groups=NOT_SET, rev='5db5d3fa364f00557fa2f', server_modified=datetime.datetime(2022, 3, 29, 15, 40, 29), sharing_info=NOT_SET, size=3289782593, symlink_info=NOT_SET)


## Evaluation

Instead of calling `trainer.test()`, we can also manually evaluate the model on the entire test set:

In [1]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

model = LUKE.load_from_checkpoint(checkpoint_path=checkpoint_callback.best_model_path)

model.to(device)

model.model.eval()

predictions_total = []
logits_total = []
labels_total = []

for batch in tqdm(model.test_dataloader()):
    # get the inputs;
    labels = batch["label"]
    del batch["label"]

    # move everything to the GPU
    for k,v in batch.items():
      batch[k] = batch[k].to('cuda')

    # forward pass
    outputs = model.model(**batch)
    logits = outputs.logits
    logits_total.extend(torch.sigmoid(logits).tolist())
    predictions = (torch.sigmoid(logits)>THRESHOLD).float()
    predictions_total.extend(predictions.tolist())
    labels_total.extend(labels.tolist())

NameError: name 'LUKE' is not defined

In [20]:
# for idx, label_name in enumerate(rel_label_names):
#     labels = [label[idx] for label in labels_total]
#     predictions = [pred[idx] for pred in predictions_total]
#     logits = [logit[idx] for logit in logits_total]
#     precision = metrics.precision_score(labels, predictions)
#     recall = metrics.recall_score(labels, predictions)
#     f1 = metrics.f1_score(labels, predictions)
#     roc_auc = metrics.roc_auc_score(labels, logits)
#     print(label_name)
#     print(f'Precision: {precision:.3f}, Recall: {recall:.3f}')
#     print(f'F1-Score: {f1:.3f}, ROC-AUC: {roc_auc:.3f}\n')

StrategicAlliance
Precision: 0.921, Recall: 0.993
F1-Score: 0.956, ROC-AUC: 0.980

JointVenture
Precision: 0.979, Recall: 0.988
F1-Score: 0.983, ROC-AUC: 0.998

Marketing
Precision: 0.821, Recall: 0.725
F1-Score: 0.770, ROC-AUC: 0.943

Licensing
Precision: 0.752, Recall: 0.859
F1-Score: 0.802, ROC-AUC: 0.967

Manufacturing
Precision: 0.816, Recall: 0.809
F1-Score: 0.812, ROC-AUC: 0.972

ResearchandDevelopment
Precision: 0.740, Recall: 0.784
F1-Score: 0.761, ROC-AUC: 0.964



## Inference

Here we test the trained model on a new, unseen sentence.

In [114]:
import torch.nn.functional as F

model.to(device)

test_doc = test_df[test_df.rels.apply(lambda rels: 'ResearchandDevelopment' in rels)].sample()

text = test_doc.document.iloc[0]
spans = test_doc.spans.iloc[0]  # character-based entity spans 

inputs = tokenizer(text, entity_spans=spans, padding="max_length", truncation=True, return_tensors="pt",
                            max_length=MAX_LEN)
inputs.to(device)

outputs = model.model(**inputs)
logits = outputs.logits
predicted_classes = (torch.sigmoid(logits)>THRESHOLD).float()
print("Sentence:", text)
print("Tokens:", tokenizer.decode(inputs["input_ids"][0]))
print("Ground truth label:", ids2labels(test_doc.rel_one_hot.iloc[0]))
print("Predicted class idx:", ids2labels(predicted_classes.squeeze().tolist()))
print("Confidence:", torch.sigmoid(logits).squeeze().tolist())

Sentence: International Business Machines Corp {IBM} and TDK Corp (TC) formed a strategic alliance to manufacture and develop high capacity Magnetic Random Access Memory (MRAM) utilizing the spin momentum transfer effect. The companies were to leverage their respective expertiese in order to create a high capacity MRAM integrated circuit which can be used as standalone memory or embedded into other IC solutions.
Tokens: <s>International Business Machines Corp { <ent> IBM <ent> } and <ent2>  TDK Corp <ent2>  (TC) formed a strategic alliance to manufacture and develop high capacity Magnetic Random Access Memory (MRAM) utilizing the spin momentum transfer effect. The companies were to leverage their respective expertiese in order to create a high capacity MRAM integrated circuit which can be used as standalone memory or embedded into other IC solutions.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [125]:
!gpustat -p

/bin/bash: gpustat: command not found
